In [1]:
# ---------------------- Imports ---------------------- #
import pandas as pd
from unidecode import unidecode
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:
# ---------------------- Load Data ------------------- #
df = pd.read_csv("../Data/FBREF_Top7LeaguesEurope_Season(2019-2024)_Uncleaned.csv")
print("Initial shape:", df.shape)
df.head()


Initial shape: (23283, 119)


,Rk,Player,Nation,Pos,Squad,Age,Born,Playing_Time_MP,Playing_Time_Starts,Playing_Time_Min,...,SCA_Types_Def,GCA,GCA90,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_TO,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Matches_gca
0,1,Mohamed Abarhoun,ma MAR,DF,Moreirense,29.0,1989.0,14,13,1171.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Matches
1,2,Amir Abedzadeh,ir IRN,GK,Marítimo,25.0,1993.0,13,13,1170.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Matches
2,3,Vincent Aboubakar,cm CMR,FW,Porto,26.0,1992.0,8,6,456.0,...,0.0,2.0,0.39,1.0,0.0,1.0,0.0,0.0,0.0,Matches
3,4,Danildo Accioly,br BRA,DF,Santa Clara,37.0,1981.0,12,10,909.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Matches
4,5,Yaw Ackah,gh GHA,MF,Boavista,19.0,1999.0,2,0,18.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Matches


In [3]:
for column in df.columns:
    print(f"{column}\n")

Rk

Player

Nation

Pos

Squad

Age

Born

Playing_Time_MP

Playing_Time_Starts

Playing_Time_Min

Playing_Time_90s

Performance_Gls

Performance_Ast

Performance_G+A

Performance_G_PK

Performance_PK

Performance_PKatt

Performance_CrdY

Performance_CrdR

Expected_xG

Expected_npxG

Expected_xAG

Expected_npxG+xAG

Progression_PrgC

Progression_PrgP

Progression_PrgR

Per_90_Minutes_Gls

Per_90_Minutes_Ast

Per_90_Minutes_G+A

Per_90_Minutes_G_PK

Per_90_Minutes_G+A_PK

Per_90_Minutes_xG

Per_90_Minutes_xAG

Per_90_Minutes_xG+xAG

Per_90_Minutes_npxG

Per_90_Minutes_npxG+xAG

Matches

Season

League

Player_shot

Nation_shot

Pos_shot

Squad_shot

Age_shot

Born_shot

90s

Standard_Gls

Standard_Sh

Standard_SoT

Standard_SoT%

Standard_Sh/90

Standard_SoT/90

Standard_G/Sh

Standard_G/SoT

Standard_Dist

Standard_FK

Standard_PK

Standard_PKatt

Expected_xG_shot

Expected_npxG_shot

Expected_npxG/Sh

Expected_G_xG

Expected_np:G_xG

Matches_shot

Player_pass

Nation_pass

Pos_pass

S

In [4]:
df.rename(columns={'1/3': 'PassesinFinalThird'}, inplace=True)

In [5]:
# ---------------------- Safe Features ------------------- #
safe_features = [
    "Pos", "Age", "Born", "Season",
    "Playing_Time_MP", "Playing_Time_Starts", "Playing_Time_Min", "Playing_Time_90s",
    "Performance_G+A", "Performance_PK", "Performance_PKatt", "Performance_CrdY", "Performance_CrdR",
    "Expected_xG", "Expected_npxG", "Expected_xAG", "Expected_npxG+xAG",
    "Progression_PrgC", "Progression_PrgP", "Progression_PrgR",
    "Per_90_Minutes_xG", "Per_90_Minutes_xAG", "Per_90_Minutes_xG+xAG", "Per_90_Minutes_npxG", "Per_90_Minutes_npxG+xAG",
    "Standard_Sh", "Standard_SoT", "Standard_SoT%", "Standard_Sh/90", "Standard_SoT/90", "Standard_FK", "Standard_Dist",
    "Expected_xG_shot", "Expected_npxG_shot", "Expected_npxG/Sh",
    "Total_Cmp", "Total_Att", "Total_Cmp%", "Total_TotDist", "Total_PrgDist",
    "Short_Cmp", "Short_Att", "Short_Cmp%",
    "Medium_Cmp", "Medium_Att", "Medium_Cmp%",
    "Long_Cmp", "Long_Att", "Long_Cmp%",
    "xAG", "Expected_xA", "KP", "PassesinFinalThird", "PPA", "CrsPA", "PrgP",
    "SCA_SCA", "SCA_SCA90", "SCA_Types_PassLive", "SCA_Types_PassDead", "SCA_Types_TO", "SCA_Types_Sh", "SCA_Types_Fld", "SCA_Types_Def"
]

rag_table_csv = [
    "Player", "Nation", "Pos", "Squad", "Age", "Born", "Season", "League",
    "Playing_Time_MP", "Playing_Time_Starts", "Playing_Time_Min", "Playing_Time_90s",
    "Performance_Gls", "Performance_Ast", "Performance_G+A", "Performance_G_PK", "Performance_PK", "Performance_PKatt", "Performance_CrdY", "Performance_CrdR",
    "Expected_xG", "Expected_npxG", "Expected_xAG", "Expected_npxG+xAG",
    "Progression_PrgC", "Progression_PrgP", "Progression_PrgR",
    "Per_90_Minutes_Gls", "Per_90_Minutes_Ast", "Per_90_Minutes_G+A", "Per_90_Minutes_G_PK", "Per_90_Minutes_G+A_PK",
    "Per_90_Minutes_xG", "Per_90_Minutes_xAG", "Per_90_Minutes_xG+xAG", "Per_90_Minutes_npxG", "Per_90_Minutes_npxG+xAG",
    "Standard_Gls", "Standard_Sh", "Standard_SoT", "Standard_SoT%", "Standard_Sh/90", "Standard_SoT/90", "Standard_G/Sh", "Standard_G/SoT", "Standard_FK", "Standard_Dist", "Standard_PK", "Standard_PKatt",
    "Expected_npxG/Sh", "Expected_G_xG", "Expected_np:G_xG",
    "Total_Cmp", "Total_Att", "Total_Cmp%", "Total_TotDist", "Total_PrgDist",
    "Short_Cmp", "Short_Att", "Short_Cmp%",
    "Medium_Cmp", "Medium_Att", "Medium_Cmp%",
    "Long_Cmp", "Long_Att", "Long_Cmp%",
    "Ast", "xAG", "Expected_xA", "Expected_A_xAG", "KP", "PassesinFinalThird", "PPA", "CrsPA", "PrgP",
    "SCA_SCA", "SCA_SCA90", "SCA_Types_PassLive", "SCA_Types_PassDead", "SCA_Types_TO", "SCA_Types_Sh", "SCA_Types_Fld", "SCA_Types_Def",
    "GCA", "GCA90", "GCA_Types_PassLive", "GCA_Types_PassDead", "GCA_Types_TO", "GCA_Types_Sh", "GCA_Types_Fld", "GCA_Types_Def"
]


In [6]:
# Create Safe Clean DataFrames
safe_df = df[safe_features].copy()
rag_table = df[rag_table_csv].copy()

print("Safe DF shape:", safe_df.shape)
print("RAG Table shape:", rag_table.shape)


Safe DF shape: (23283, 64)
RAG Table shape: (23283, 91)


In [7]:
# Fill Missing Percentage Columns
# List of percentage columns to fill with 0
percent_cols = [
    "Standard_SoT%",
    "Expected_npxG/Sh",
    "Total_Cmp%",
    "Standard_Dist",
    "Short_Cmp%",
    "Medium_Cmp%",
    "Long_Cmp%"
]

# Fill missing values with 0 ONLY for these columns
for col in percent_cols:
    if col in safe_df.columns:
        safe_df[col] = safe_df[col].fillna(0)

for col in percent_cols:
    if col in rag_table.columns:
        rag_table[col] = rag_table[col].fillna(0)

In [8]:
# Drop Remaining NA in Safe DF
safe_df.dropna(inplace=True)
print("Safe DF after dropna:", safe_df.shape)


Safe DF after dropna: (23281, 64)


In [9]:
# Clean Player Names
# Function to clean player names (remove special chars, accents, etc.)
def clean_player_name(name):
    # normalize accents / special characters
    name = unidecode(name.strip())
    # remove punctuation except space and hyphen
    name = re.sub(r"[^a-zA-Z\s\-]", "", name)
    # split and join to remove extra spaces
    parts = name.split()
    return " ".join(parts)

In [10]:
# Function to clean nation column (remove lowercase letters before country code)
def clean_nation(nation):
    if pd.isna(nation):
        return nation
    # Remove lowercase letters followed by space, keep only the uppercase country code
    # Pattern: remove any lowercase letters at the start followed by space
    cleaned = re.sub(r'^[a-z]+\s+', '', nation.strip())
    return cleaned


In [11]:
# Function to clean squad/club names
def clean_squad_name(squad):
    if pd.isna(squad):
        return squad
    # normalize accents / special characters
    squad = unidecode(squad.strip())
    # remove punctuation except space and hyphen
    squad = re.sub(r"[^a-zA-Z0-9\s\-]", "", squad)
    # split and join to remove extra spaces
    parts = squad.split()
    return " ".join(parts)


In [12]:
# Apply cleaning to Player column in rag_table
rag_table['Player'] = rag_table['Player'].apply(clean_player_name)

# Apply cleaning to Nation column in rag_table
rag_table['Nation'] = rag_table['Nation'].apply(clean_nation)

# Apply cleaning to Squad column in rag_table
rag_table['Squad'] = rag_table['Squad'].apply(clean_squad_name)

In [13]:
# Rename Squad column to Club
rag_table.rename(columns={'Squad': 'Club'}, inplace=True)

In [14]:
print("Safe DF final shape:", safe_df.shape)
print("RAG Table final shape:", rag_table.shape)

Safe DF final shape: (23281, 64)
RAG Table final shape: (23283, 91)


In [15]:
# Save cleaned RAG table
rag_table.to_csv("../Data/rag_table_csv_cleaned.csv", index=False)
print("Saved cleaned RAG table!")

Saved cleaned RAG table!


In [16]:
# Display sample of cleaned data
print("\nSample of cleaned RAG table:")
print(rag_table.head())
print("\nSample of cleaned safe_df:")
print(safe_df.head())


Sample of cleaned RAG table:
              Player Nation Pos         Club   Age    Born  Season  \
0   Mohamed Abarhoun    MAR  DF   Moreirense  29.0  1989.0    2019   
1     Amir Abedzadeh    IRN  GK     Maritimo  25.0  1993.0    2019   
2  Vincent Aboubakar    CMR  FW        Porto  26.0  1992.0    2019   
3    Danildo Accioly    BRA  DF  Santa Clara  37.0  1981.0    2019   
4          Yaw Ackah    GHA  MF     Boavista  19.0  1999.0    2019   

          League  Playing_Time_MP  Playing_Time_Starts  ...  SCA_Types_Fld  \
0  Primeira Liga               14                   13  ...            0.0   
1  Primeira Liga               13                   13  ...            0.0   
2  Primeira Liga                8                    6  ...            0.0   
3  Primeira Liga               12                   10  ...            0.0   
4  Primeira Liga                2                    0  ...            1.0   

   SCA_Types_Def  GCA  GCA90  GCA_Types_PassLive  GCA_Types_PassDead  \
0       

In [17]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 1. Select object columns
obj_cols = safe_df.select_dtypes(include=['object']).columns

# 2. Convert them to string (important!)
safe_df[obj_cols] = safe_df[obj_cols].astype(str)

# 3. OneHotEncoder — dense output, ignore unknown values
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", dtype=int)

# 4. Fit & transform
ohe_array = ohe.fit_transform(safe_df[obj_cols])

# 5. Convert to DataFrame with correct column names
ohe_df = pd.DataFrame(ohe_array, columns=ohe.get_feature_names_out(obj_cols))

# 6. Drop original object columns
merged_numeric = safe_df.drop(columns=obj_cols).reset_index(drop=True)

# 7. Concatenate numeric + encoded columns
merged_numeric = pd.concat([merged_numeric, ohe_df], axis=1)

merged_numeric


,Age,Born,Season,Playing_Time_MP,Playing_Time_Starts,Playing_Time_Min,Playing_Time_90s,Performance_G+A,Performance_PK,Performance_PKatt,...,Pos_DF,"Pos_DF,FW","Pos_DF,MF",Pos_FW,"Pos_FW,DF","Pos_FW,MF",Pos_GK,Pos_MF,"Pos_MF,DF","Pos_MF,FW"
0,29.0,1989.0,2019,14,13,1171.0,13.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,25.0,1993.0,2019,13,13,1170.0,13.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,26.0,1992.0,2019,8,6,456.0,5.1,5.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,37.0,1981.0,2019,12,10,909.0,10.1,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,19.0,1999.0,2019,2,0,18.0,0.2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23276,18.0,2004.0,2024,6,0,59.0,0.7,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
23277,18.0,2004.0,2024,16,8,773.0,8.6,4.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
23278,25.0,1998.0,2024,26,15,1387.0,15.4,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
23279,19.0,2004.0,2024,12,3,492.0,5.5,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [18]:
# ---------------------- Train on 2019–22, Test on 2023–24 ---------------------- #
train_df = merged_numeric[merged_numeric["Season"] <= 2022]
test_df  = merged_numeric[merged_numeric["Season"] >= 2023]

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (15375, 73)
Test shape: (7906, 73)


In [19]:
X_train = train_df.drop("Performance_G+A", axis=1)
y_train = train_df["Performance_G+A"]

X_test  = test_df.drop("Performance_G+A", axis=1)
y_test  = test_df["Performance_G+A"]

In [20]:
# ---------------------- Scaling ---------------------- #
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [21]:
# ---------------------- Models ---------------------- #
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    "SVR": SVR(kernel='rbf', C=10, gamma=0.1),
    "KNN": KNeighborsRegressor(n_neighbors=5)
}


In [22]:
# ---------------------- Model Evaluation ---------------------- #
results = []

for name, model in models.items():

    if name in ["Linear Regression", "Ridge Regression", "SVR", "KNN"]:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    })

results_df = pd.DataFrame(results).sort_values(by="R2", ascending=False)

print("\n================ FINAL RESULTS (Train: 2019-22, Test: 2023-24) ================")
print(results_df)


================ FINAL RESULTS (Train: 2019-22, Test: 2023-24) ================
               Model       MAE      RMSE        R2
1   Ridge Regression  0.875512  1.366125  0.908155
0  Linear Regression  0.875810  1.366597  0.908091
3            XGBoost  0.882084  1.424260  0.900172
2      Random Forest  0.888838  1.429281  0.899466
5                KNN  1.036934  1.680886  0.860956
4                SVR  2.043607  3.181803  0.501778


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import numpy as np


In [24]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor  = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)


In [25]:
class MLP_Model(nn.Module):
    def __init__(self, input_dim, h1=512, h2=256, h3=128, h4=64, dropout=0.3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.BatchNorm1d(h1),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(h2, h3),
            nn.BatchNorm1d(h3),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(h3, h4),
            nn.BatchNorm1d(h4),
            nn.GELU(),

            nn.Linear(h4, 1)
        )

    def forward(self, x):
        return self.net(x)


In [26]:
def train_and_evaluate_improved(h1, h2, h3, h4, dropout, lr):

    model = MLP_Model(
        input_dim=X_train.shape[1],
        h1=h1, h2=h2, h3=h3, h4=h4,
        dropout=dropout
    )

    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    # Cosine learning rate scheduler (excellent for deep MLPs)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=20, eta_min=lr/10
    )

    best_loss = float('inf')
    patience_counter = 0
    patience = 12

    for epoch in range(200):  # deep nets converge faster with CosineAnnealing
        model.train()
        epoch_loss = 0

        for X_batch, y_batch in train_loader:
            preds = model(X_batch)
            loss = criterion(preds, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(X_test_tensor), y_test_tensor).item()

        scheduler.step()

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    # Final predictions
    model.eval()
    with torch.no_grad():
        final_preds = model(X_test_tensor).flatten()

    mae = mean_absolute_error(y_test, final_preds)
    rmse = root_mean_squared_error(y_test, final_preds)
    r2 = r2_score(y_test, final_preds)

    return mae, rmse, r2, model


In [27]:
param_grid = [
    (512, 256, 128, 64, 0.3, 0.001),
    (1024, 512, 256, 128, 0.3, 0.001),
    (768, 384, 192, 96, 0.25, 0.0007),
    (512, 512, 256, 128, 0.2, 0.0005),
    (128, 64, 32, 16, 0.2, 0.0005),
]


results = []

for h1, h2, h3, h4, dp, lr in param_grid:
    mae, rmse, r2, model = train_and_evaluate_improved(h1, h2, h3, h4, dp, lr)
    results.append(((h1, h2, h3, dp, lr), mae, rmse, r2))

results


[((512, 256, 128, 0.3, 0.001),
  0.9409963029057217,
  1.453573880703083,
  0.8960199237511568),
 ((1024, 512, 256, 0.3, 0.001),
  0.8939331351566655,
  1.4019933680412415,
  0.9032685207867066),
 ((768, 384, 192, 0.25, 0.0007),
  0.8960701022876717,
  1.420085833882167,
  0.9007558078833624),
 ((512, 512, 256, 0.2, 0.0005),
  0.9223034205620785,
  1.4178946304154894,
  0.9010618407134081),
 ((128, 64, 32, 0.2, 0.0005),
  0.89595508886807,
  1.3979425863877641,
  0.9038266860976238)]

In [28]:
best_params = max(results, key=lambda x: x[3])  # smallest MAE
best_params


((128, 64, 32, 0.2, 0.0005),
 0.89595508886807,
 1.3979425863877641,
 0.9038266860976238)